In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

$X= ``Número\ de\ gols\ do\ time\ de\ casa" \sim Poisson(\lambda_x)\\$
$Y= ``Número\ de\ gols\ do\ time\ visitante" \sim Poisson(\lambda_x)$

$$P(X=x)=\frac{\lambda^{x}_{x}e^{-\lambda_x}}{x!}\qquad P(Y=y)=\frac{\lambda^{y}_{y}e^{-\lambda_y}}{y!}\\$$

$$P(X=x, Y=y)=P(X=x)P(Y=y)=\frac{\lambda_{x}^{x}\lambda_{y}^{y}e^{-(\lambda_x+\lambda_y)}}{x!y!}$$

In [45]:
tabela = pd.read_excel('dados/tabela/tabela-rodada_26.xlsx')
historico_casa = pd.read_excel('dados/historico/historicocasa-rodada_26.xlsx')
historico = pd.read_excel('dados/historico/historico-rodada_26.xlsx')
rodadas = pd.read_excel('dados/previsoes/todas_rodadas.xlsx')

rodada_atual = tabela['MP'].max()
rodadas_futuras = rodadas[rodadas['gols_fora'].isna()]
#rodadas_antigas = 

In [46]:
rodadas = rodadas.loc[rodadas['Home'].notna()]
rodadas_futuras = rodadas[rodadas['gols_fora'].isna()]

In [47]:
def preparar_estatisticas():
  #ATT Rating = (Gols Feitos no campeonato/Partidas Jogadas)/gols_liga
  #DEF Rating = (Gols Sofridos no campeonato/Partidas Jogadas)/gols_liga
  #EXPECTED GOALS = ATTxDEFxAVGLeague

  gols_feitos = tabela['GF']/tabela['MP']
  gols_tomados = tabela['GA']/tabela['MP']
  gols_liga = tabela['GF']/tabela['MP'].mean()

  att_rating = gols_feitos/gols_liga
  def_rating = gols_tomados/gols_liga

  tabela['expG'] = att_rating*def_rating*gols_liga

  #xG e Gols, fora e dentro de casa
  for time in tabela['Squad']:
    tabela.loc[tabela['Squad'] == time, 'xG_casa'] = historico_casa[historico_casa['Equipe'] == time]['xG'].mean()
    tabela.loc[tabela['Squad'] == time, 'xG_fora'] = historico_casa[historico_casa['Opponent'] == time]['xGA'].mean()
    tabela.loc[tabela['Squad'] == time, 'xG_total'] = tabela['xG_casa'] + tabela['xG_fora']
    tabela.loc[tabela['Squad'] == time, 'GF_casa'] = historico_casa[historico_casa['Equipe'] == time]['GF'].mean()
    tabela.loc[tabela['Squad'] == time, 'GF_fora'] = historico_casa[historico_casa['Opponent'] == time]['GA'].mean()
    tabela.loc[tabela['Squad'] == time, 'GF_total'] = tabela['GF_casa'] + tabela['GF_fora']
    tabela.loc[tabela['Squad'] == time, 'GA_casa'] = historico_casa[historico_casa['Equipe'] == time]['GA'].mean()
    tabela.loc[tabela['Squad'] == time, 'GA_fora'] = historico_casa[historico_casa['Opponent'] == time]['GF'].mean()
    tabela.loc[tabela['Squad'] == time, 'GA_total'] = tabela['GA_casa'] + tabela['GA_fora']
    tabela.loc[tabela['Squad'] == time, 'xG_sofrido_casa'] = historico_casa[historico_casa['Equipe'] == time]['xGA'].mean()
    tabela.loc[tabela['Squad'] == time, 'xG_sofrido_fora'] = historico_casa[historico_casa['Opponent'] == time]['xG'].mean()
    

  #Ultimos 5 jogos, xG e Gols, fora e dentro de casa
  historico_5jogos = historico_casa[historico_casa['Round'] > rodada_atual-5]

  for time in tabela['Squad']:
    tabela.loc[tabela['Squad'] == time, 'xG_5casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['xG'].mean()
    tabela.loc[tabela['Squad'] == time, 'xG_5fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['xGA'].mean()
    tabela.loc[tabela['Squad'] == time, 'xG_5total'] = tabela['xG_casa'] + tabela['xG_fora']
    tabela.loc[tabela['Squad'] == time, 'GF_5casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['GF'].mean()
    tabela.loc[tabela['Squad'] == time, 'GF_5fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['GA'].mean()
    tabela.loc[tabela['Squad'] == time, 'GF_5total'] = tabela['GF_5casa'] + tabela['GF_5fora']
    tabela.loc[tabela['Squad'] == time, 'GA_5casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['GA'].mean()
    tabela.loc[tabela['Squad'] == time, 'GA_5fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['GF'].mean()
    tabela.loc[tabela['Squad'] == time, 'GA_5total'] = tabela['GA_5casa'] + tabela['GA_5fora']
    tabela.loc[tabela['Squad'] == time, 'xG_sofrido_casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['xGA'].mean()
    tabela.loc[tabela['Squad'] == time, 'xG_sofrido_fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['xG'].mean()

In [48]:
def gerar_lambda(par_x, par_y, peso_x, peso_y, rodada):
  df = rodadas_futuras[rodadas_futuras['Wk'] == rodada]
  dem_x = (sum(peso_x[0])+sum(peso_x[1]))
  dem_y = (sum(peso_y[0])+sum(peso_y[1]))

  for time_casa, time_fora in zip(df['Home'], df['Away']):
    # Time alvo == Casa
    lambda_x0 = sum(tabela.loc[tabela['Squad'] == time_casa, par_x[0][i]]*peso_x[0][i] for i in range(len(par_x[0])))
    lambda_x1 = sum(tabela.loc[tabela['Squad'] == time_fora, par_x[1][i]]*peso_x[1][i] for i in range(len(par_x[1])))
    lambda_x = (float(lambda_x0) + float(lambda_x1))/dem_x
    # Time alvo == Casa
    lambda_y0 = sum(tabela.loc[tabela['Squad'] == time_fora, par_y[0][i]]*peso_y[0][i] for i in range(len(par_y[0])))
    lambda_y1 = sum(tabela.loc[tabela['Squad'] == time_casa, par_y[1][i]]*peso_y[1][i] for i in range(len(par_y[1])))
    lambda_y = (float(lambda_y0) + float(lambda_y1))/dem_y
    
    df.loc[df['Home'] == time_casa, 'lambda_x'] = float(lambda_x)
    df.loc[df['Away'] == time_fora, 'lambda_y'] = float(lambda_y)

  return df

In [49]:
def prob_vitoria(rodada_df, rodada, lambdas):
  rodada_df = rodada_df.loc[rodada_df['Wk'] == rodada]
  prob_gol = {}
  times_casa = []
  times_fora = []
  
  # Times casa
  for time in rodada_df['Home']:
    prob_gol[time] = []
    times_casa.append(time)
    for k in range(0, 16):
      prob_gol[time].append(stats.poisson.pmf(k, lambdas.loc[lambdas['Home'] == time, 'lambda_x']))
  
  # Times Fora
  for time in rodada_df['Away']:
    prob_gol[time] = []
    times_fora.append(time)
    for k in range(0, 16):
      prob_gol[time].append(stats.poisson.pmf(k, lambdas.loc[lambdas['Away'] == time, 'lambda_y']))

  casa_vitoria = {}
  fora_vitoria = {}

  for time_casa, time_fora in zip(times_casa, times_fora): 
    casa_vitoria[time_casa] = []
    fora_vitoria[time_fora] = []
    for i in range(1, 11):
      casa_vitoria[time_casa].append(prob_gol[time_casa][i] * sum(prob_gol[time_fora][:i]))
      fora_vitoria[time_fora].append(prob_gol[time_fora][i] * sum(prob_gol[time_casa][:i]))
    casa_vitoria[time_casa] = sum(casa_vitoria[time_casa])
    fora_vitoria[time_fora] = sum(fora_vitoria[time_fora])
    rodada_df.loc[rodada_df['Home'] == time_casa, 'vitoria_casa'] = casa_vitoria[time_casa]
    rodada_df.loc[rodada_df['Away'] == time_fora, 'vitoria_fora'] = fora_vitoria[time_fora]
    rodada_df.loc[rodada_df['Away'] == time_fora, 'empate'] = 1- (rodada_df.loc[rodada_df['Home'] == time_casa, 'vitoria_casa'] + rodada_df.loc[rodada_df['Away'] == time_fora, 'vitoria_fora'])
    

  return rodada_df, prob_gol

In [50]:
def prob_gols(rodadas_prob, prob_gol):
  rodadas_probgols = pd.DataFrame.from_dict(prob_gol, orient='index').astype(float)

  df_casa = pd.DataFrame()
  for time in rodadas_prob['Home']:
    linha_casa = pd.DataFrame(rodadas_probgols.loc[time,]).T
    linha_casa['Home'] = time
    df_casa = pd.concat([df_casa, linha_casa])

  df_fora = pd.DataFrame()
  for time in rodadas_prob['Away']:
    linha_fora = pd.DataFrame(rodadas_probgols.loc[time,]).T
    linha_fora['Away'] = time
    df_fora = pd.concat([df_fora, linha_fora])


  df_qsefinal = pd.merge(rodadas_prob, df_casa, on='Home')
  df_qsefinal = pd.merge(df_qsefinal, df_fora, on='Away')
  df_final = pd.merge(rodadas_futuras, df_qsefinal, on= ['Wk', 'Day', 'Date', 'Home', 'xGCasa', 'gols_casa', 'gols_fora', 'xGFora', 'Away'], how='left')


  return df_final, df_qsefinal

In [53]:
par_x = [['xG_total', 'xG_casa', 'GF_total', 'GF_casa', 'xG_5total', 'xG_5casa', 'GF_5total', 'GF_5casa'], #Ataque casa
        ['GA_total', 'GA_fora', 'xG_total', 'xG_fora', 'xG_5total', 'xG_5fora', 'GA_5total', 'GA_5fora']]  #Defesa fora
pes_x = [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]


par_y = [['xG_total', 'xG_fora', 'GF_total', 'GF_fora', 'xG_5total', 'xG_5fora', 'GF_5total', 'GF_5fora'], #Ataque fora
        ['GA_total', 'GA_casa', 'xG_total', 'xG_casa', 'xG_5total', 'xG_5casa', 'GA_5total', 'GA_5casa']]  #Defesa casa
pes_y = [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [54]:
preparar_estatisticas()

In [55]:
lambdas = gerar_lambda(par_x, par_y, pes_x, pes_y, rodada_atual+1)

In [56]:
rodadas_prob, prob_gol = prob_vitoria(rodadas_futuras, rodada_atual+1, lambdas)

In [57]:
rodadas_futuras_prob, rodada_futura_prob = prob_gols(rodadas_prob, prob_gol)

In [59]:
rodadas_futuras_prob.to_excel('dados/previsoes/todas_previsoes.xlsx')
rodada_futura_prob.to_excel(f'dados/previsoes/previsao_rodada-{rodada_atual+1}.xlsx')